In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install dependencies
!pip install web3==6.15.0 -q
print("✅ Setup complete!")

Mounted at /content/drive
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.4/355.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.7 MB/s eta 0:00:00
✅ Setup complete!


In [ ]:
import json
from web3 import Web3

# Load your student data
with open('/content/drive/MyDrive/student_data.json', 'r') as f:
    data = json.load(f)

students = [Web3.to_checksum_address(addr) for addr in data['students']]
hashes = data['hashes']

print(f"✅ Loaded {len(students)} students + {len(hashes)} hashes")
print(f"First student: {students[0]}")
print(f"First hash:    {hashes[0][:20]}...")

✅ Loaded 20000 students + 20000 hashes
First student: 0xef913C0BB17d0f37d7FDd895b84283824Dd444DA
First hash:    Qm19076634b8f9dd4649...


In [ ]:
# ⚠️ CRITICAL: REPLACE WITH YOUR VALUES
PRIVATE_KEY = "1f46fa8e940853a20ff25a5b1eac62e22deee08bdd71902eafdb39102fb6ee5b"  # Export from MetaMask
WALLET_ADDRESS = "0x9bB27cbcf567521f96c8d180310eC69415372f6C"  # Your address
CONTRACT_ADDRESS = "0xd9145CCE52D386f254917e481eB44e9943F39138"  # Your deployed contract

# Connect to Polygon Amoy
w3 = Web3(Web3.HTTPProvider('https://rpc-amoy.polygon.technology/'))
account = w3.eth.account.from_key(PRIVATE_KEY)
w3.eth.default_account = account.address

print(f"✅ Connected to Polygon Amoy")
print(f"Wallet: {account.address}")
print(f"Balance: {w3.from_wei(w3.eth.get_balance(account.address), 'ether')} MATIC")


✅ Connected to Polygon Amoy
Wallet: 0x9bB27cbcf567521f96c8d180310eC69415372f6C
Balance: 0.1 MATIC


In [ ]:
# Cell 3B: CHECK BALANCE
balance_wei = w3.eth.get_balance(account.address)
balance_matic = w3.from_wei(balance_wei, 'ether')
print(f"💰 Balance: {balance_matic} MATIC")
print(f"✅ READY" if balance_matic > 0.1 else "❌ NEED 0.1+ MATIC")


💰 Balance: 0.1 MATIC
❌ NEED 0.1+ MATIC


In [ ]:
# abi
abi=json.loads('''[
	{
		"inputs": [
			{
				"internalType": "address[]",
				"name": "_students",
				"type": "address[]"
			},
			{
				"internalType": "string[]",
				"name": "_hashes",
				"type": "string[]"
			}
		],
		"name": "batchRegister",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"name": "registry",
		"outputs": [
			{
				"internalType": "string",
				"name": "ipfsHash",
				"type": "string"
			},
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
''')

In [ ]:
# Use your EXISTING contract with CORRECT ABI
CONTRACT_ADDRESS = "0xd9145CCE52D386f254917e481eB44e9943F39138"
contract = w3.eth.contract(address=Web3.to_checksum_address(CONTRACT_ADDRESS), abi=abi)

print(f"✅ Connected to existing contract: {CONTRACT_ADDRESS}")
print("💡 Ready for registration!")


✅ Connected to existing contract: 0xd9145CCE52D386f254917e481eB44e9943F39138
💡 Ready for registration!


In [ ]:
# Deploy fresh contract
# ⚠️ CRITICAL: REPLACE WITH YOUR COMPILED CONTRACT BYTECODE (e.g., from a .bin file or compilation output)
bytecode = "0x6080604052348015600e575f5ffd5b506105c58061001c5f395ff3fe608060405234801561000f575f5ffd5b5060043610610034575f3560e01c8063038defd7146100385780632a022a0c14610062575b5f5ffd5b61004b6100463660046101c9565b610077565b6040516100599291906101e9565b60405180910390f35b61007561007036600461037a565b610118565b005b5f602081905290815260409020805481906100919061043d565b80601f01602080910402602001604051908101604052809291908181526020018280546100bd9061043d565b80156101085780601f106100df57610100808354040283529160200191610108565b820191905f5260205f20905b8154815290600101906020018083116100eb57829003601f168201915b5050505050908060010154905082565b5f5b82518110156101a957604051806040016040528083838151811061014057610140610475565b60200260200101518152602001428152505f5f85848151811061016557610165610475565b6020908102919091018101516001600160a01b031682528101919091526040015f208151819061019590826104d4565b50602091909101516001918201550161011a565b505050565b80356001600160a01b03811681146101c4575f5ffd5b919050565b5f602082840312156101d9575f5ffd5b6101e2826101ae565b9392505050565b604081525f83518060408401528060208601606085015e5f606082850101526060601f19601f8301168401019150508260208301529392505050565b634e487b7160e01b5f52604160045260245ffd5b604051601f8201601f1916810167ffffffffffffffff8111828210171561026257610262610225565b604052919050565b5f67ffffffffffffffff82111561028357610283610225565b5060051b60200190565b5f82601f83011261029c575f5ffd5b81356102af6102aa8261026a565b610239565b8082825260208201915060208360051b8601019250858311156102d0575f5ffd5b602085015b8381101561037057803567ffffffffffffffff8111156102f3575f5ffd5b8601603f81018813610303575f5ffd5b602081013567ffffffffffffffff81111561032057610320610225565b610333601f8201601f1916602001610239565b8181526040838301018a1015610347575f5ffd5b816040840160208301375f602083830101528086525050506020830192506020810190506102d5565b5095945050505050565b5f5f6040838503121561038b575f5ffd5b823567ffffffffffffffff8111156103a1575f5ffd5b8301601f810185136103b1575f5ffd5b80356103bf6102aa8261026a565b8082825260208201915060208360051b8501019250878311156103e0575f5ffd5b6020840193505b82841015610409576103f8846101ae565b8252602093840193909101906103e7565b9450505050602083013567ffffffffffffffff811115610427575f5ffd5b6104338582860161028d565b9150509250929050565b600181811c9082168061045157607f821691505b60208210810361046f57634e487b7160e01b5f52602260045260245ffd5b50919050565b634e487b7160e01b5f52603260045260245ffd5b601f8211156101a957805f5260205f20601f840160051c810160208510156104ae5750805b601f840160051c820191505b818110156104cd575f81556001016104ba565b5050505050565b815167ffffffffffffffff8111156104ee576104ee610225565b610502816104fc845461043d565b84610489565b6020601f821160018114610534575f831561051d5750848201515b5f19600385901b1c1916600184901b1784556104cd565b5f84815260208120601f198516915b828110156105635787850151825560209485019460019092019101610543565b508482101561058057868401515f19600387901b60f8161c191681555b50505050600190811b0190555056fea2646970667358221220dbc8b1f563dd509388ede1ae91e522b4e3c97b758a42e628f79345e4c3157b0064736f6c634300081f0033"
StudentRegistry = w3.eth.contract(abi=abi, bytecode=bytecode)

txn = StudentRegistry.constructor().build_transaction({
    'from': account.address,
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 3000000,
    'gasPrice': w3.to_wei('30', 'gwei'),
    'chainId': 80002
})

signed = w3.eth.account.sign_transaction(txn, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction) # ✅ Correct for Web3.py v6+
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

contract_address = receipt.contractAddress
contract = w3.eth.contract(address=contract_address, abi=abi)

print(f"🎉 DEPLOYED at: {contract_address}")
print(f"TX: {tx_hash.hex()}")
print(f"✅ Verify: https://amoy.polygonscan.com/address/{contract_address}")

🎉 DEPLOYED at: 0x8784Ae5D73100D908F4fB1A0E33218f4bA96b490
TX: 0xf158b4a074e1f971598bb9d96c13e6d4af4a747f44c6d80b57ec3ff612cd0c6a
✅ Verify: https://amoy.polygonscan.com/address/0x8784Ae5D73100D908F4fB1A0E33218f4bA96b490


In [ ]:
# This will print every function currently recognized by your ABI
print([func.fn_name for func in contract.all_functions()])

['batchRegister', 'registry']


In [ ]:
# Updated test for single registration using the BATCH function
def test_single_registration():
    # Wrapping single items in lists [] to match batchRegister signature
    txn = contract.functions.batchRegister([students[0]], [hashes[0]]).build_transaction({
        'from': account.address,
        'nonce': w3.eth.get_transaction_count(account.address),
        'gas': 500000, # Increased gas slightly for batch logic
        'gasPrice': w3.to_wei('30', 'gwei'),
        'chainId': 80002
    })

    signed = w3.eth.account.sign_transaction(txn, PRIVATE_KEY)

    # Using 'rawTransaction' for Web3.py v6+ compatibility
    tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    print(f"✅ Student 1 registered via batchRegister!")
    print(f"TX Hash: {tx_hash.hex()}")
    return receipt.status == 1

success = test_single_registration()
print("Transaction Status: ✅" if success else "Transaction Status: ❌")

✅ Student 1 registered via batchRegister!
TX Hash: 0x085a525530a0fde6996c1dc5f0185312153fb8b5d97cf395e8234c50f9a64842
Transaction Status: ✅


In [ ]:
balance_wei = w3.eth.get_balance("0x9bB27cbcf567521f96c8d180310eC69415372f6C")
print(f"💰 Balance: {w3.from_wei(balance_wei, 'ether')} POL")

💰 Balance: 0.04039837 POL


In [ ]:
# Register 100 students (5 batches of 20)
def bulk_register_students(max_students=100, batch_size=20):
    success_count = 0

    for i in range(0, min(max_students, len(students)), batch_size):
        batch_students = students[i:i+batch_size]
        batch_hashes = hashes[i:i+batch_size]

        try:
            txn = contract.functions.batchRegister(batch_students, batch_hashes).build_transaction({
                'from': account.address,
                'nonce': w3.eth.get_transaction_count(account.address),
                'gas': 3000000,  # Increased gas limit
                'gasPrice': w3.to_wei('1', 'gwei'),  # Lowered gas price to reduce transaction cost
                'chainId': 80002
            })

            signed = w3.eth.account.sign_transaction(txn, PRIVATE_KEY)
            tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
            receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

            if receipt.status == 1:
                success_count += len(batch_students)
                print(f"✅ Batch {i//batch_size+1}: {len(batch_students)} students")
            else:
                print(f"❌ Batch {i//batch_size+1} failed")

        except Exception as e:
            print(f"❌ Batch {i//batch_size+1} error: {e}")

    print(f"\n🎉 TOTAL: {success_count} students registered!")
    return success_count

bulk_register_students(100)

❌ Batch 1 error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 40398370000000000, tx cost 75000000000000000, overshot 34601630000000000'}
❌ Batch 2 error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 40398370000000000, tx cost 75000000000000000, overshot 34601630000000000'}
❌ Batch 3 error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 40398370000000000, tx cost 75000000000000000, overshot 34601630000000000'}
❌ Batch 4 error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 40398370000000000, tx cost 75000000000000000, overshot 34601630000000000'}
❌ Batch 5 error: {'code': -32000, 'message': 'insufficient funds for gas * price + value: balance 40398370000000000, tx cost 75000000000000000, overshot 34601630000000000'}

🎉 TOTAL: 0 students registered!


0

In [ ]:
def verify_registrations(sample_size=5):
    print("🔍 VERIFYING REGISTRATIONS")
    registered = 0

    for i in range(sample_size):
        student = students[i]
        try:
            ipfs_hash, timestamp = contract.functions.getStudentRecord(student).call()
            if timestamp > 0:
                print(f"✅ {student[:12]}... | {ipfs_hash[:20]}... | {timestamp}")
                registered += 1
            else:
                print(f"❌ {student[:12]}... | Not registered")
        except Exception as e:
            print(f"❌ {student[:12]}... | Error: {e}")

    print(f"\n📊 {registered}/{sample_size} verified!")
    return registered == sample_size

verify_registrations(10)
